In [203]:
from IPython.display import HTML
from pathlib import Path

import jinja2 as jj
import numpy as np
import pandas as pd
import requests as rq

In [204]:
def do_dev_tasks(html):
    file_url = '.local'
    
    if Path(file_url).is_dir():
        with open(f'{file_url}/index.html', 'w') as f:
            f.write(html)


def get_dataframe(name):
    data_url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
               f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    
    return pd.read_csv(data_url)


def get_css_asset(name):
    data_url = f'https://raw.githubusercontent.com/iamashks/covid19-australia/master/assets/css/{name}.css'
    file_url = f'assets/css/{name}.css'
    
    if Path(file_url).is_file():
        asset = f'<link rel="stylesheet" type="text/css" href="../{file_url}" />\n'
    else:
        asset = f'<style>{rq.get(data_url).text}</style>'
    
    return asset


def get_template(name):
    data_url = f'https://raw.githubusercontent.com/iamashks/covid19-australia/master/templates/{name}.html'
    file_url = f'templates/{name}.html'
    
    if Path(file_url).is_file():
        templateLoader = jj.FileSystemLoader(searchpath='./')
        templateEnv = jj.Environment(loader=templateLoader)
        template = templateEnv.get_template(file_url)
    else:
        template = jj.Template(rq.get(data_url).text)
    
    return template

In [205]:
COL_COUNTRY = 'Country/Region'
COL_STATE = 'Province/State'
COUNTRY = 'Australia'

dft_confirm = get_dataframe('confirmed')
dft_confirm = dft_confirm[dft_confirm[COL_COUNTRY] == COUNTRY]
 
dft_demised = get_dataframe('deaths')
dft_demised = dft_demised[dft_demised[COL_COUNTRY] == COUNTRY]

dft_recover = get_dataframe('recovered')
dft_recover = dft_recover[dft_recover[COL_COUNTRY] == COUNTRY]

In [206]:
COL_TODAY = dft_confirm.columns[-1]
COL_1DAY = dft_confirm.columns[-1 - 1]
COL_5DAY = dft_confirm.columns[-1 - 5]
COL_50DAY = dft_confirm.columns[-1 - 50]

df_table = pd.DataFrame({'State': dft_confirm[COL_STATE], 'Cases': dft_confirm[COL_TODAY], 
                         'Recover': dft_recover[COL_TODAY], 'Deaths': dft_demised[COL_TODAY]})

df_table['Cases (5D)'] = dft_confirm[COL_TODAY] - dft_confirm[COL_5DAY]
df_table['Recover (5D)'] = dft_recover[COL_TODAY] - dft_recover[COL_5DAY]
df_table['Deaths (5D)'] = dft_demised[COL_TODAY] - dft_demised[COL_5DAY]
df_table['Cases (1D)'] = dft_confirm[COL_TODAY] - dft_confirm[COL_1DAY]
df_table['Recover (1D)'] = dft_recover[COL_TODAY] - dft_recover[COL_1DAY]
df_table['Deaths (1D)'] = dft_demised[COL_TODAY] - dft_demised[COL_1DAY]
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)

df_table = df_table.sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
df_table = df_table.reset_index()
df_table.index += 1

del df_table['index'] # del duplicate index
df_table.head(8)

,State,Cases,Recover,Deaths,Cases (5D),Recover (5D),Deaths (5D),Cases (1D),Recover (1D),Deaths (1D),Fatality Rate
1,New South Wales,3059,2571,45,12,85,1,6,28,1,1.5
2,Victoria,1514,1385,18,47,58,0,3,25,0,1.2
3,Queensland,1052,1028,6,7,34,0,1,1,0,0.6
4,Western Australia,553,537,9,1,3,0,0,1,0,1.6
5,South Australia,439,434,4,0,1,0,0,0,0,0.9
6,Tasmania,227,184,13,0,10,0,0,2,0,5.7
7,Australian Capital Territory,107,104,3,0,1,0,0,0,0,2.8
8,Northern Territory,29,27,0,0,0,0,0,0,0,0.0


In [207]:
dt_cols = dft_confirm.columns[~dft_confirm.columns.isin([COL_STATE, COL_COUNTRY, 'Lat', 'Long'])]
dft_cases = dft_confirm.groupby(COL_STATE)[dt_cols].sum()
dft_cases_new = dft_cases.diff(axis=1).fillna(0).astype(int)

include_cols = ['Cases', 'Recover', 'Deaths', 'Cases (5D)', 'Recover (5D)', 'Deaths (5D)']

summary_nsw = df_table[df_table['State'].eq('New South Wales')][include_cols].sum().add_prefix('NSW ')
summary_vic = df_table[df_table['State'].eq('Victoria')][include_cols].sum().add_prefix('VIC ')
summary_qld = df_table[df_table['State'].eq('Queensland')][include_cols].sum().add_prefix('QLD ')

summary_time = {'updated': pd.to_datetime(COL_TODAY), 'since': pd.to_datetime(COL_5DAY)}
summary = {**summary_time, **df_table[include_cols].sum(), **summary_nsw, **summary_vic, **summary_qld}

summary['Active'] = summary['Cases'] - summary['Recover'] - summary['Deaths']
summary['Active (5D)'] = summary['Cases (5D)'] - summary['Recover (5D)'] - summary['Deaths (5D)']
summary

{'updated': Timestamp('2020-05-12 00:00:00'),
 'since': Timestamp('2020-05-07 00:00:00'),
 'Cases': 6980,
 'Recover': 6270,
 'Deaths': 98,
 'Cases (5D)': 67,
 'Recover (5D)': 192,
 'Deaths (5D)': 1,
 'NSW Cases': 3059,
 'NSW Recover': 2571,
 'NSW Deaths': 45,
 'NSW Cases (5D)': 12,
 'NSW Recover (5D)': 85,
 'NSW Deaths (5D)': 1,
 'VIC Cases': 1514,
 'VIC Recover': 1385,
 'VIC Deaths': 18,
 'VIC Cases (5D)': 47,
 'VIC Recover (5D)': 58,
 'VIC Deaths (5D)': 0,
 'QLD Cases': 1052,
 'QLD Recover': 1028,
 'QLD Deaths': 6,
 'QLD Cases (5D)': 7,
 'QLD Recover (5D)': 34,
 'QLD Deaths (5D)': 0,
 'Active': 612,
 'Active (5D)': -126}

In [208]:
html_text = get_template('overview').render(D=summary, table=df_table, newcases=dft_cases_new, 
                                            np=np, pd=pd, enumerate=enumerate)

html_text = f'<div>{get_css_asset("keen")}{html_text}</div>'

do_dev_tasks(html=html_text)
HTML(html_text)
